<a href="https://colab.research.google.com/github/kandgesid/Data-Science-and-ML-projects/blob/master/malware_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !unzip '/content/drive/MyDrive/malware_detection_project/malimg_dataset.zip' -d '/content/drive/MyDrive/malware_detection_project'

In [ ]:
# !unzip '/content/drive/MyDrive/malware_detection_eproject/benign _file.zip' -d '/content/drive/MyDrive/malware_detection_project'

In [ ]:
import tensorflow as tf

In [ ]:
import numpy as np
from math import sqrt, ceil
import cv2
import os
from tensorflow.keras.preprocessing import imageex
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix,accuracy_score
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from collections import Counter
import numpy as np
np.random.seed(1)
import glob
import time

In [ ]:
tf.__version__

'2.4.1'

In [ ]:
# Dataset loading and spliting in train and test folder on drive..
import shutil
# !unzip '/content/drive/MyDrive/malware_detection_project/malimg_dataset.zip' -d '/content/drive/MyDrive/malware_detection_project'
# !unzip '/content/drive/MyDrive/malware_detection_project/benign _file.zip' -d '/content/drive/MyDrive/malware_detection_project'
path_root = "/content/drive/MyDrive/malware_detection_project/malimg_paper_dataset_imgs"
for folder in os.listdir(path_root):
  train_path = os.path.join('/content/drive/MyDrive/malware_detection_project/malware_dataset/train', folder)
  os.mkdir(train_path)
  ori_path = os.path.join(path_root, folder)
  images_list = os.listdir(ori_path)
  print(images_list)
  length = len(images_list)
  print("length of original file:", length)
  train_ratio = length // 10
  count = 0
  img_count = train_ratio * 7
  print("count of file in train folder", img_count)
  for img_file in images_list:
    if count <= img_count:
      print("hello")
      img_path = os.path.join(ori_path, img_file)
      shutil.move(img_path, train_path)
      count += 1
    else:
      break

In [ ]:
def process_path(path, model):
  if os.path.isdir(path):
    for item in os.listdir(path):
      if os.path.isfile(item):
        # call is_malicoius function
        # store result to result dictionary along with path as key.(join(path and item))
        file_path = os.path.join(path, item)
        res = is_malicious(file_path, model)
        result[file_path] = res
      elif os.path.isdir(item):
        dir_path = os.path.join(path, item)
        process_path(dir_path)
        # call process_path function on item
      else:
        print("File/directory not exist..")
        exit()
  else:
    # call is_malicious on item
    if os.path.isfile(path):
      res = is_malicious(path, model)
      result[path] = res
    else:
      print("File not exist..")
      exit()

In [ ]:
list_families = ['Adialer.C', 'Agent.FYI', 'Allaple.A', 'Allaple.L', 'Alueron.gen!J', 'Autorun.K', 'C2LOP.P', 'C2LOP.gen!g', 'Dialplatform.B', 'Dontovo.A', 'Fakerean', 'Instantaccess', 'Lolyda.AA1', 'Lolyda.AA2', 'Lolyda.AA3', 'Lolyda.AT', 'Malex.gen!J', 'Obfuscator.AD', 'Rbot!gen', 'Skintrim.N', 'Swizzor.gen!E', 'Swizzor.gen!I', 'VB.AT', 'Wintrim.BX', 'Yuner.A', 'benign']
def is_malicious(file, model):
  img = convert_to_img(file)
  img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
  img = cv2.resize(img, (224, 224))
  img_batch = np.expand_dims(img, axis=0)
  prediction = new_model.predict(img_batch)
  f_name = list_families[np.argmax(prediction)]
  return f_name


In [ ]:
def convert_to_img(file):
  with open(file, 'rb') as binary_file:
      data = binary_file.read()

  # Data length in bytes
  data_len = len(data)

  # d is a verctor of data_len bytes
  d = np.frombuffer(data, dtype=np.uint8)

  # Assume image shape should be close to square
  sqrt_len = int(ceil(sqrt(data_len)))  # Compute square toot and round up

  # Requiered length in bytes.
  new_len = sqrt_len*sqrt_len

  # Number of bytes to pad (need to add zeros to the end of d)
  pad_len = new_len - data_len

  # Pad d with zeros at the end.
  # padded_d = np.pad(d, (0, pad_len))
  padded_d = np.hstack((d, np.zeros(pad_len, np.uint8)))

  # Reshape 1D array into 2D array with sqrt_len pad_len x sqrt_len (im is going to be a Grayscale image).
  im = np.reshape(padded_d, (sqrt_len, sqrt_len))
  #print(im.shape)
  name = "converted_img"+".png"
  #print(im.shape)
  cv2.imwrite(name, im)
  return im


In [ ]:
# Converts benign files to gray scale images and store on drive
# %cd '/content/drive/MyDrive/malware_detection_project/benign'
# from google.colab.patches import cv2_imshow
# path = "/content/drive/MyDrive/malware_detection_project/benign_files"
# for i, file in enumerate(os.listdir(path)):
#   f = os.path.join(path , file)
#   convert_to_img(f)

In [ ]:
# create generators for training and testing dataset
root_path = "/content/drive/MyDrive/malware_detection_project/malware_dataset"
TRAIN_DIR = os.path.join(root_path, "train")
TEST_DIR = os.path.join(root_path, "test")

data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

train_generator = data_gen_train.flow_from_directory(
	TRAIN_DIR,
	target_size=(224, 224),
	class_mode='categorical',
  batch_size=512
)

validation_generator = data_gen_valid.flow_from_directory(
	TEST_DIR,
	target_size=(224, 224),
	class_mode='categorical',
  batch_size=512,
  shuffle=False
  )

In [ ]:
train_generator.class_indices

In [ ]:
list_families = list(train_generator.class_indices.keys())
print(list_families)

In [ ]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Input,Flatten,Dense,Dropout,GlobalAveragePooling2D,Conv2D,MaxPooling2D
num_classes = 26
image_shape = ((224, 224, 3))
base_model = tf.keras.applications.VGG16(weights='imagenet', input_shape=image_shape, include_top=False)
base_model.trainable = False
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
fc1 = Dense(160, activation='relu')(global_average_layer)
fc1 = Dropout(0.3)(fc1)
fc2 = Dense(160, activation='relu')(fc1)
fc2 = Dropout(0.3)(fc2)
prediction_layer = Dense(num_classes, activation='softmax')(fc2)
model = Model(inputs=base_model.input, outputs=prediction_layer)

58892288/58889256 [==============================] - 0s 0us/step


In [ ]:
# Assigned weights to each target class to mitigate imbalance from dataset
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(
               'balanced',
                np.unique(train_generator.classes),
                train_generator.classes)
CLASS_WEIGHTS = {}
for i, weight in enumerate(class_weights):
  CLASS_WEIGHTS[i] = weight

In [ ]:
from collections import Counter
itemCt = Counter(train_generator.classes)
maxCt = float(max(itemCt.values()))
cw = {clsID : maxCt/numImg for clsID, numImg in itemCt.items()}

In [ ]:
CLASS_WEIGHTS

In [ ]:
early_stopping = EarlyStopping(verbose=1, patience=10, monitor='val_loss')
checkpoint_path = '/content/drive/MyDrive/malware_detection_project/checkpoints_VGG16_test4/training_check_1/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)
checkpointer = ModelCheckpoint(filepath=checkpoint_dir, monitor='loss', verbose=0, save_best_only=True, save_weights_only=True, mode='min')
callbacks_list = [early_stopping, checkpointer]

In [ ]:
# prediction model training process
num_epochs = 100
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_generator, validation_data=validation_generator, epochs=num_epochs, steps_per_epoch=7222//512 ,verbose=1, callbacks=callbacks_list, class_weight=CLASS_WEIGHTS)
# batch_size = 256

In [ ]:
plt.plot(model.history.history['accuracy'])
plt.plot(model.history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(model.history.history['loss'])
plt.plot(model.history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
probabilities = model.predict_generator(generator=validation_generator)
y_true = validation_generator.classes
y_pred = probabilities > 0.5
font = {
'family': 'Times New Roman',
'size': 12
}
matplotlib.rc('font', **font)
mat = confusion_matrix(y_true, y_pred)
plot_confusion_matrix(conf_mat=mat, figsize=(8, 8), show_normed=False)

In [ ]:
latest = tf.train.latest_checkpoint('/content/drive/MyDrive/malware_detection_project/checkpoints_VGG16_test4')
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.load_weights(latest)
# score = model.evaluate(validation_generator)
# print(score)

In [ ]:
saved_model_path = '/content/drive/MyDrive/malware_detection_project/saved_model/VGG16_model_test1'
new_model = tf.keras.models.load_model(saved_model_path)
#score = new_model.evaluate(validation_generator)

In [ ]:
new_model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [ ]:
tf.keras.utils.plot_model(
    new_model,
    to_file="model.png",
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
)

In [ ]:
result = {}
process_path('/content/vbc.exe.config', new_model)

In [ ]:
result

{'/content/vbc.exe.': 'benign', '/content/vbc.exe.config': 'benign'}

In [ ]:
# required list to plot statistics
family_counts = Counter(result.values())
family_name = [f_name for f_name in family_counts.keys()]
occurences = [num for num in family_counts.values()]

In [ ]:
img_path = '/content/drive/MyDrive/malware_detection_project/malware_dataset/test/Swizzor.gen!I/06caeea27451023fca6debca71933975.png'
# img = convert_to_img('/content/vbc.exe.config')
# img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
# img = cv2.resize(img, (244, 224))
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_batch = np.expand_dims(img_array, axis=0)
ans = new_model.predict(img_batch)

In [ ]:
print(list_families[np.argmax(ans)])

In [ ]:
result